# Notebook Memory Networks - Question Answering

### The dataset can be downloaded - https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz

In [38]:
# import the necessary packages

from keras.layers import Input
from keras.layers.core import Dense, Activation, Dropout, Permute
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.models import Model
import collections
import nltk

import numpy as np
import itertools
import matplotlib.pyplot as plt

%matplotlib inline

### Read the files to create train and test data

In [33]:
TRAIN_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt"
TEST_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt"

stories = []
ques = []
answers = []
def parseFile(infile):
    story = []
    # This parses each line and creates the stories, questions and answers tuple
    for line in open(infile):
        content = line.split(" ", 1)
#         print(content[1])
        if "\t" in line:
            stor_ques_ans = content[1].split("\t")
            ques.append(stor_ques_ans[0])
            answers.append(stor_ques_ans[1])
            stories.append(story)
            story = []
        else:
            story.append(content[1])
#     print(ques)
    return stories, ques, answers
    
train_data = parseFile(TRAIN_FILE)
test_data = parseFile(TEST_FILE)

In [39]:
# swap the subscript for the train_data and test_data to 0 or 1 or 2 to get the Stories, Questions or Answers

print(train_data[1])
print(" ")
print(test_data[1])

['Where is Mary? ', 'Where is Daniel? ', 'Where is Daniel? ', 'Where is Daniel? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is John? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Mary? ', 'Where is John? ', 'Where is John? ', 'Where is Sandra? ', 'Where is Daniel? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Mary? ', 'Where is John? ', 'Where is Sandra? ', 'Where is John? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Daniel? ', 'Where is Mary? ', 'Where is Sandra? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Daniel? ', 'Where is Mary? ', 'Where is John? ', 'Where is Daniel? ', 'Where is John? ', 'Where is Sandra? ', 'Where is Daniel? ', 'Where is Daniel? ', 'Where is Daniel? ', 'Where is Sandra? ', 'Where is Daniel? ', 'Where is Sandra? ', 'Where is Sandra? ', 'Where is John? ', 'Where is Sandra? ', 'Where

### Build Vocab from traina and test data

In [72]:
# this method returns the word2index and index2word mappings 
word_counts = collections.Counter()

def buildVocab(train_data, test_data):
    # this loop runs through all the sentences , questions and answers and creates a counter with the counts for 
    # unique words and their counts.
    for stories, questions, answers in [train_data, test_data]:
        for sent in stories:
            for sen in sent:
                for word in nltk.word_tokenize(sen):
                    word_counts[word] += 1
        for ques in questions:
            for word in nltk.word_tokenize(ques):
                word_counts[word] += 1
        for ans in answers:
            for word in nltk.word_tokenize(ans):
                word_counts[word] += 1
#     print(len(word_counts))
#     print(" ")
#     print(word_counts)
    
    # now create the word2index and index2word maps
    i=0
    word2index = {w:i+1 for i,w in enumerate(word_counts.keys())}
#     print(" ")
#     print(len(word2index))
#     print(word2index)
    
    index2word = {v:k for k,v in word2index.items()}
#     print(" ")
#     print(len(index2word))
#     print(index2word)
    return word2index, index2word

word2index, index2word = buildVocab(train_data, test_data)
        

### Get Maximum Lengths for the story and Question

### Vectorize the Inputs and Pad the sequences

### Define the Model Architecture